In [1]:
import pandas as pd
import datetime
from WindPy import w
import requests
from Portfolio import portfolio
import time
import matplotlib.pyplot as plt
import easytrader
import pprint
import math

In [2]:
def rounddown(x, n):
    return int(math.floor(x / n)) * n

# 1. Connect to Wind and 华泰

In [3]:
# start WInd
w.start()

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2020 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


.ErrorCode=0
.Data=[OK!]

In [17]:
# start 华泰
user = easytrader.use('ht_client')
user.prepare(user='666637575127', password='671109', comm_password='246810')

# 2. Get the current position

## Set up position template
## Our desired funds
### 1. 160325.SZ 华夏创业 2022/7/24
### 2. 160926.SZ 创业大成 2022/7/16
### 3. 501080.SH 科创中金 2022/7/11
### 4. 501082.SH 科创投资 2022/6/27
### 5. 506002.SH 易基科创 2022/7/28
### 6. 506003.SH 富国科创 2022/7/29
### 7. 506006.SH 添富科创 2022/7/28

In [8]:
position = {
    "160325":{
        "name":"华夏创业",
        "ticker":"160325.SZ",
        "price":w.wsq('160325.SZ', 'rt_last').Data[0][0],
        "shares":0,
        "expiration":(datetime.datetime(2022, 7, 24, 0, 0, 0) - datetime.datetime.now()).days/365
    }, 
    "160926":{
        "name":"创业大成",
        "ticker":"160926.SZ",
        "price":w.wsq('160926.SZ', 'rt_last').Data[0][0],
        "shares":0,
        "expiration":(datetime.datetime(2022, 7, 16, 0, 0, 0) - datetime.datetime.now()).days/365
    }, 
    "501080":{
        "name":"科创中金",
        "ticker":"501080.SH",
        "price":w.wsq('501080.SH', 'rt_last').Data[0][0],
        "shares":0,
        "expiration":(datetime.datetime(2022, 7, 11, 0, 0, 0) - datetime.datetime.now()).days/365
    },
    "501082":{
        "name":"科创投资",
        "ticker":"501082.SH",
        "price":w.wsq('501082.SH', 'rt_last').Data[0][0],
        "shares":0,
        "expiration":(datetime.datetime(2022, 6, 27, 0, 0, 0) - datetime.datetime.now()).days/365
    },
    "506002":{
        "name":"易基科创",
        "ticker":"506002.SH",
        "price":w.wsq('506002.SH', 'rt_last').Data[0][0],
        "shares":0,
        "expiration":(datetime.datetime(2022, 7, 28, 0, 0, 0) - datetime.datetime.now()).days/365
    },
    "506003":{
        "name":"富国科创",
        "ticker":"506003.SH",
        "price":w.wsq('506003.SH', 'rt_last').Data[0][0],
        "shares":0,
        "expiration":(datetime.datetime(2022, 7, 29, 0, 0, 0) - datetime.datetime.now()).days/365
    },
    "506006":{
        "name":"添富科创",
        "ticker":"506006.SH",
        "price":w.wsq('506006.SH', 'rt_last').Data[0][0],
        "shares":0,
        "expiration":(datetime.datetime(2022, 7, 28, 0, 0, 0) - datetime.datetime.now()).days/365
    },
}

## Read 华泰 position and balance

In [9]:
ht_position = user.position

targets = [
    '506002',
    '501082',
    '506006',
    '160926',
    '506003',
    '160325',
    '501080'
]

for holding in ht_position:
    if holding['证券代码'] in targets:
        position[holding['证券代码']]['shares'] = round(holding['市值']/holding['市价'], -2)

In [10]:
balance = user.balance

In [11]:
naive = portfolio(cash=balance['可用金额'], position=position)
fee = 0.0003

In [12]:
def strategy(interval, portfolio, threshold):
    
    cur_times = 0
    
    log_df = pd.DataFrame()
    
    dt = datetime.datetime.now()
    
    while dt.time() < datetime.time(15):
    
        print(dt.time())
        now = datetime.datetime.now()
        
        # 1. create snapshot to document price and NAV
        snapshot = {now : {}}
        for code, info in portfolio.position.items():
            
            headers = {
                'content-type': 'application/json',
                'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:22.0) Gecko/20100101 Firefox/22.0'
            }
            response = requests.get(f'https://fundgz.1234567.com.cn/js/{code}.js', headers=headers)
            NAV = float(eval(response.text.replace('jsonpgz(', '').replace(');', ''))['gsz'])
            
            Price = w.wsq(info['ticker'], 'rt_last').Data[0][0]
            info['price'] = Price
            AnnualizedRateOfReturn = round(((NAV/Price)**(1/info['expiration'])-1)*100, 2)
            #NAV, Price, AnnualizedRateOfReturn
            snapshot[now][code + ' NAV'] = NAV
            snapshot[now][code + ' Price'] = Price
            snapshot[now][code + ' AnnualizedRateOfReturn (%)'] = AnnualizedRateOfReturn
        
        in_ranks = {k:v for k,v in snapshot[now].items() if "AnnualizedRateOfReturn (%)" in k and portfolio.position[k.split(' ')[0]]['shares'] != 0}
        out_ranks = {k:v for k,v in snapshot[now].items() if "AnnualizedRateOfReturn (%)" in k and portfolio.position[k.split(' ')[0]]['shares'] == 0}
        in_ranks = {k.split(' ')[0]:snapshot[now][k] for k in sorted(in_ranks, key=in_ranks.get, reverse=True)}        
        out_ranks = {k.split(' ')[0]:snapshot[now][k] for k in sorted(out_ranks, key=out_ranks.get, reverse=True)}
        print("Holdings: ")
        print(in_ranks)
        print("Not in holdings: ")
        print(out_ranks)
            
        # 2. Trade
        if list(out_ranks.items())[0][1] - list(in_ranks.items())[-1][1] >= threshold:
            print(f"Sell {list(in_ranks.items())[-1][0]}")
            print(f"Buy {list(out_ranks.items())[0][0]}")
            
            sale = list(in_ranks.items())[-1][0]
            purchase = list(out_ranks.items())[0][0]
            
            sell_result = user.sell(sale, price=portfolio.position[sale]['price']-0.001, amount=portfolio.position[sale]['shares'])
            print(sell_result)
            
            purchase_amount = rounddown((user.balance['可用金额'] * fee)/(portfolio.position[purchase]['price']+0.001), 100)
            buy_result = user.buy(purchase, price=portfolio.position[purchase]['price']+0.001, amount=purchase_amount)
            print(buy_result)
            
        ht_position = user.position

        targets = [
            '506002',
            '501082',
            '506006',
            '160926',
            '506003',
            '160325',
            '501080'
        ]
 
        for holding in ht_position:
            if holding['证券代码'] in targets:
                position[holding['证券代码']]['shares'] = round(holding['市值']/holding['市价'], -2)
                
        portfolio.position = position
        portfolio.cash = user.balance['可用金额']
        
        time.sleep(interval)
        dt = datetime.datetime.now()

    return log_df

In [14]:
log_df = strategy(60, naive, threshold=1)

14:58:51.147600
Holdings: 
{'506002': 13.86, '160926': 13.81, '501082': 13.81, '506003': 13.76}
Not in holdings: 
{'160325': 13.43, '501080': 12.69, '506006': 12.25}
14:59:57.329600
Holdings: 
{'160926': 13.81, '501082': 13.8, '506002': 13.79, '506003': 13.79}
Not in holdings: 
{'160325': 13.51, '501080': 12.72, '506006': 12.15}


In [ ]:
dt = datetime.datetime.now()

In [ ]:
dt.time()

In [18]:
user.today_trades

[{'成交日期': 20210809,
  '成交时间': '13:21:09',
  '证券代码': '506003',
  '证券名称': '富国科创',
  '操作': '买入',
  '成交数量': 640,
  '成交均价': 0.978,
  '成交金额': 625.92,
  '交易市场': '上海Ａ',
  '股东帐户': 'A360822222',
  '成交编号': 148381,
  '合同编号': '49943',
  'Unnamed: 12': ''},
 {'成交日期': 20210809,
  '成交时间': '13:21:09',
  '证券代码': '506003',
  '证券名称': '富国科创',
  '操作': '买入',
  '成交数量': 25600,
  '成交均价': 0.978,
  '成交金额': 25036.8,
  '交易市场': '上海Ａ',
  '股东帐户': 'A360822222',
  '成交编号': 148380,
  '合同编号': '49943',
  'Unnamed: 12': ''},
 {'成交日期': 20210809,
  '成交时间': '13:21:09',
  '证券代码': '506003',
  '证券名称': '富国科创',
  '操作': '买入',
  '成交数量': 260,
  '成交均价': 0.978,
  '成交金额': 254.28,
  '交易市场': '上海Ａ',
  '股东帐户': 'A360822222',
  '成交编号': 148379,
  '合同编号': '49943',
  'Unnamed: 12': ''},
 {'成交日期': 20210809,
  '成交时间': '13:19:11',
  '证券代码': '506006',
  '证券名称': '添富科创',
  '操作': '卖出',
  '成交数量': 21302,
  '成交均价': 1.18,
  '成交金额': 25136.36,
  '交易市场': '上海Ａ',
  '股东帐户': 'A360822222',
  '成交编号': 147427,
  '合同编号': '49656',
  'Unnamed: 12': ''},
 {'成交日期': 20210809

In [23]:
user.refresh()